In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Sieć neuronowa

[Sieci neuronowe](https://pl.wikipedia.org/wiki/Sie%C4%87_neuronowa) składają się z warstw neuronów połączonych ze sobą. Każda warstwa składa się z neuronów takich jak implementowaliśmy w poprzednim zestawie, jednak wydajniej jest ją zaimplementować jako całość niż składać z pojedynczych obiektów. Warstwa neuronów posiada macierz wag $W$ oraz wektor biasów $\overline{b}$.
Wagi należy początkowo zainicjalizować do losowych wartości. Najczęściej losuje się je z rozkładu normalnego o zerowej średniej i jednostkowej wariancji. Z kolei początkowe biasy można przyjąć w zasadzie dowolne (mogą to być małe liczby losowe).

Do zaimplementowania warstwy wystarczą 3 podstawowe funkcje:

Forward pass - propagujemy dane od wejścia warstwy do wyjścia

Backward pass - propagujemy gradient od wyjścia do wejścia

Parameter update - etap w którym wyliczamy gradienty wag i biasów (etap uczenia neuronów)


# Neuron network

[Neural networks] (https://en.wikipedia.org/wiki/Artificial_neural_network) consist of layers of neurons connected to each other. Each layer consists of neurons as we implemented in the previous set, but it is more efficient to implement it as a whole than to assemble it from individual objects. The neuron layer has the $W$ weight matrix and the $\overline{b}$ bias vector.
Initially, weights should be initialized to random values. They are most often drawn from the normal distribution with zero mean and unit variance. In turn, the initial bias can be assumed to be basically any value(they can be small random numbers).

To implement the layer, 3 basic functions are sufficient:

Forward pass - we propagate data from layer entry to exit

Backward pass - we propagate the gradient from exit to entry

Parameter update - a stage where we calculate weight and bias gradients (neuron learning step)

In [2]:
class Layer:
    
    def __init__(self, numberOfNeurons, numberOfInputs):
        self.numberOfInputs = numberOfInputs
        self.numberOfNeurons = numberOfNeurons
        self.weights = None
        self.bias = None
        
        self.tmp_input = None
        self.tmp_gradient = None
        self.tmp_output = None
        ### BEGIN SOLUTION
        self.weights = np.random.randn(numberOfNeurons, numberOfInputs) / np.sqrt(numberOfInputs)
        self.bias = np.random.uniform(0, 0.1, numberOfNeurons)
        ### END SOLUTION
        
    def forward(self, inputVector):
        ### BEGIN SOLUTION
        self.tmp_output = np.dot(self.weights, inputVector) + self.bias
        self.tmp_output = 1.0 / (1.0 + np.exp(-self.tmp_output))
        self.tmp_input = inputVector
        return self.tmp_output
        ### END SOLUTION
        
    def backward(self, gradient):
        ### BEGIN SOLUTION
        activation_gradient = self.tmp_output * (1.0 - self.tmp_output)
        self.tmp_gradient = gradient * activation_gradient
        output = np.dot(np.transpose(self.weights), self.tmp_gradient)
        return output
        ### END SOLUTION
        
    def learn(self, learningRate):
        ### BEGIN SOLUTION
        weight_update = np.tensordot(self.tmp_gradient, self.tmp_input, axes=0)
        bias_update = self.tmp_gradient
        
        self.weights += learningRate * weight_update
        self.bias += learningRate * bias_update
        ### END SOLUTION


### Forward pass

Warstwa neuronów wylicza swoją aktywację (wektor na wyjściu) w następujący sposób.

$$ \overline{y}_{out}=f_{act}(W \cdot \overline{x}_{in}+\overline{b})$$

gdzie $\cdot$ to iloczyn skalarny, $W$ to macierz wag neuronu, $\overline{x}_{in}$ to wektor danych wejściowych, $\overline{b}$ to wektor biasów, a $f_{act}$ to pewna nieliniowa funkcja aktywacyjna. Można wybrać wiele różnych takich funkcji, w tym zestawie także użyjemy funkcji sigmoidalnej
$$f_{act}(x)=sigm(x)=\frac{1}{1+e^{-x}}$$

Należy zwrócić uwagę, żeby funkcja $forward$ zwracała np.array. Ponadto należy skopiować wektor wejściowy do self.tmp_input, oraz zapamiętać wynik w self.tmp_output. Będzie to potrzebne w następnych zadaniach.

### Forward pass

The neuron layer calculates its activation (output vector) as follows.

$$ \overline{y}_{out}=f_{act}(W \cdot \overline{x}_{in}+\overline{b})$$

where $\cdot$ is a scalar product, $W$ is a neuron weight matrix, $\overline{x}_{in}$ is an input vector, $\overline{b}$ is a bias vector, and $f_{act}$ is a nonlinear activation function. You can choose many different such functions, and in this set we will also use a sigmoidal function
$$f_{act}(x)=sigm(x)=\frac{1}{1+e^{-x}}$$

Make sure $forward$ returns np.array. In addition, copy the input vector to self.tmp_input, and save the result in self.tmp_output. You will need it in the next tasks.

In [3]:
n = Layer(7, 4)
n.weights = np.reshape(np.arange(0, 2.8, 0.1), (7, 4))
n.bias = np.array(np.arange(-0.3, 0.4, 0.1))

inp = np.array((1, 2, 3, -4))
out = n.forward(inp)

np.testing.assert_array_almost_equal(out, np.array([0.3318, 0.5498, 0.7502, 0.8807, 0.9478, 0.9781, 0.9909]), decimal=3)
np.testing.assert_equal(type(out), np.ndarray)
np.testing.assert_equal(type(out[0]), np.float64)
np.testing.assert_equal(out.shape, (7,))
np.testing.assert_equal(n.tmp_input, inp)
np.testing.assert_equal(n.tmp_output, out)

### BEGIN HIDDEN TESTS
n.weights = np.random.rand(7, 4)
n.bias = np.random.rand(7)
inp = np.random.rand(4)
out = n.forward(inp)
correct = 1.0 / (1.0 + np.exp(-np.dot(n.weights, inp) - n.bias))
np.testing.assert_array_almost_equal(out, correct, decimal=3)
### END HIDDEN TESTS

## Backward pass

W tej części [propagujemy gradient wstecz](https://en.wikipedia.org/wiki/Backpropagation) według następującego wzoru

$$ \overline{g}_{out}=W^T \cdot(\overline{g}_{in} \odot \overline{f'}_{act})$$

Gdzie $\overline{g}_{in}$ to gradient który dociera do warstwy (argument funkcji $backward$), $W$ to macierz wag, a $\overline{f'}_{act}$ to wektor pochodnych funkcji aktywacyjnych. Operator $\odot$ to iloczyn Hadamarda. Funkcja sigmoidalna ma przydatną własność $sigm(x)'=sigm(x)*(1-sigm(x))$, w której należy wykorzystać zapamiętaną wartość $\overline{y}_{out}$. Wynik $\overline{g}_{in} \odot \overline{f'}_{act}$ należy zapamiętać jako self.tmp_gradient.

## Backward pass

In this part [we propagate a gradient back] (https://en.wikipedia.org/wiki/Backpropagation) according to the following formula

$$ \overline{g}_{out}=W^T \cdot(\overline{g}_{in} \odot \overline{f'}_{act})$$

Where $\overline{g}_{in}$ is the gradient that reaches the layer ($backward$ function argument), $W$ is the weight matrix, and $\overline{f'}_{act}$ is the vector of derived activation functions. The $\odot$ operator is the product of Hadamard. The sigmoid function has the useful property of $sigm(x)'=sigm(x)*(1-sigm(x))$, in which the stored value of $\overline{y}_{out}$ should be used. The $\overline{g}_{in} \odot \overline{f'}_{act}$ result should be saved as self.tmp_gradient.

In [4]:
n = Layer(7, 4)
n.weights = np.reshape(np.arange(0, 2.8, 0.1), (7, 4))
n.bias = np.array(np.arange(-0.3, 0.4, 0.1))
n.tmp_output = np.array([0.3318, 0.5498, 0.7502, 0.8807, 0.9478, 0.9781, 0.9909])

grad = np.array((-4, 2, 1, -2, 1, 0, -1))
prev = n.backward(grad)

np.testing.assert_array_almost_equal(prev, np.array([0.153, 0.116, 0.078, 0.041]), decimal=3)
np.testing.assert_equal(type(prev), np.ndarray)
np.testing.assert_equal(type(prev[0]), np.float64)
np.testing.assert_equal(prev.shape, (4,))

### BEGIN HIDDEN TESTS
n.weights = np.random.rand(7, 4)
n.bias = np.random.rand(7)
inp = np.random.rand(4)
grad = np.random.rand(7)
out = n.forward(inp)
prev = n.backward(grad)

correct = np.dot(n.weights.T, grad * (1.0 - out) * out)
np.testing.assert_array_almost_equal(prev, correct, decimal=3)
### END HIDDEN TESTS

## Update

Gradienty ze względu na wagi oraz bias:
$$ \Delta W = (\overline{g}_{in} \odot \overline{f'}_{act}) \cdot \overline{x}_{in}^T$$
$$ \Delta \overline{b} = \overline{g}_{in} \odot \overline{f'}_{act} $$
Przydaje się tutaj zapamiętana wartość self.tmp_input oraz self.tmp_gradient

Mając gradienty ze względu na wagi i bias $ \Delta \overline{w}$, $ \Delta b$ można wyliczyć ich nowe wartości, które nieco zmniejszą błąd.

$$W_{i+1} = W_{i}+\alpha  \Delta W$$
$$\overline{b}_{i+1} = \overline{b}_{i}+\alpha  \Delta \overline{b}$$

Powyższe wzory odpowiadają metodzie SGD z poprzednich zestawów.


## Update

Gradients due to weight and bias:
$$ \Delta W = (\overline{g}_{in} \odot \overline{f'}_{act}) \cdot \overline{x}_{in}^T$$
$$ \Delta \overline{b} = \overline{g}_{in} \odot \overline{f'}_{act} $$
The self.tmp_input and self.tmp_gradient values are useful here

Given the weight and bias gradients, $ \Delta \overline{w}$ and $ \Delta b$ can calculate their new values that will slightly reduce the error.

$$W_{i+1} = W_{i}+\alpha  \Delta W$$
$$\overline{b}_{i+1} = \overline{b}_{i}+\alpha  \Delta \overline{b}$$

The above formulas correspond to the SGD method from previous notebooks.

In [5]:
n = Layer(7, 4)
n.weights = np.zeros((7, 4))
n.bias = np.zeros(7)
n.tmp_output = np.array([0.3318, 0.5498, 0.7502, 0.8807, 0.9478, 0.9781, 0.9909])
n.tmp_gradient = np.array([-0.8868, 0.4950, 0.1873, -0.2101, 0.0494, 0.0, -0.0090])
n.tmp_input = np.array((1, 2, 3, -4))
n.learn(1)

correct_delta_w = np.array(
[[-0.8868, -1.7736, -2.6604, 3.5472],
 [ 0.495 ,  0.99  ,  1.485 , -1.98  ],
 [ 0.1873,  0.3746,  0.5619, -0.7492],
 [-0.2101, -0.4202, -0.6303,  0.8404],
 [ 0.0494,  0.0988,  0.1482, -0.1976],
 [ 0.    ,  0.    ,  0.    , -0.    ],
 [-0.009 , -0.018 , -0.027 ,  0.036 ]])

np.testing.assert_array_almost_equal(n.weights, correct_delta_w, decimal=3)
np.testing.assert_array_almost_equal(n.bias, n.tmp_gradient, decimal=3)

### BEGIN HIDDEN TESTS
n.weights = np.zeros((7, 4))
n.bias = np.zeros(7)
n.tmp_gradient = np.random.rand(7)
n.tmp_input = np.random.rand(4)
n.learn(1)

correct_dw = np.outer(n.tmp_gradient, n.tmp_input)
np.testing.assert_array_almost_equal(n.weights, correct_dw, decimal=3)
np.testing.assert_array_almost_equal(n.bias, n.tmp_gradient, decimal=3)
### END HIDDEN TESTS